# CAMEL Model for all the cooperatives

## Functions

In [1]:
import pandas as pd

### Capital Adequacy

In [2]:
# Loss of Assets
def loss_of_assets(excel_file, company_name, year, month):
    date_ = pd.Timestamp(f'{int(year)}-{int(month)}-01')
    
    mask1 = (excel_file['CUENTA'] == 300000) & (excel_file['fecha'] == date_)
    mask2 = (excel_file['CUENTA'] == 310000) & (excel_file['fecha'] == date_)
    
    account_coop1 = excel_file.loc[mask1, company_name]
    account_coop2 = excel_file.loc[mask2, company_name]
    
    if account_coop1.empty or account_coop2.empty:
        return 0

    value1 = list(account_coop1)[0]
    value2 = list(account_coop2)[0]

    return value1 / value2 if value2 != 0 else 0

# Solvency Ratio
def solvency_ratio():
    return 1

# Indicator of the relationship between the non-reducible minimum social contributions and the Share Capital
def min_social_contrib_social_capital(excel_file, company_name, year, month):
    date_ = pd.Timestamp(f'{int(year)}-{int(month)}-01')
    
    mask1 = (excel_file['CUENTA'] == 311000) & (excel_file['fecha'] == date_)
    mask2 = (excel_file['CUENTA'] == 310000) & (excel_file['fecha'] == date_)
    
    account_coop1 = excel_file.loc[mask1, company_name]
    account_coop2 = excel_file.loc[mask2, company_name]

    if account_coop1.empty or account_coop2.empty:
        return 0
    
    value1 = list(account_coop1)[0]
    value2 = list(account_coop2)[0]
    
    return value1 / value2 if value2 != 0 else 0


# --- Sum the values of every row 'i' in a specific company
def summation(excel_file, index_list, company_name, year, month):
    suma = 0
    date_ = pd.Timestamp(f'{int(year)}-{int(month)}-01')
    for i in index_list:
        mask = (excel_file['CUENTA'] == i) & (excel_file['fecha'] == date_)
        account_coop = excel_file.loc[mask, company_name]
        if account_coop.empty or account_coop.isna().all():
            value = 0
        else:
            value = account_coop.iloc[0]  # o .sum() si esperas múltiples filas
        suma += value
    return suma


# Indicator of the relationship between Institutional Capital and Total Assets
def capital_contribution_ratio(excel_file, company_name, year, month):
    date_ = pd.Timestamp(f'{int(year)}-{int(month)}-01')
    capital_inst_indices = [320000, 330000, 340000]
    capital_inst = summation(excel_file, capital_inst_indices, company_name, year, month)
    
    mask_total_assets = (excel_file['CUENTA'] == 100000) & (excel_file['fecha'] == date_)
    total_assets = excel_file.loc[mask_total_assets, company_name]
    if total_assets.empty:
        return 0
    total_assets_ = list(total_assets)
    value1 = capital_inst
    value2 = total_assets_[0]

    return value1 / value2 if value2 != 0 else 0

### Assets Quality

In [3]:
# Functions for quality indicator by risk with penalties
def gross_portfolio(excel_file, company_name, year, month):
    gross_portfolio_indices = [
        140400, 140500, 141100, 141200, 144100, 144200,
        144800, 145400, 145500, 146100, 146200, 146900
    ]
    return summation(excel_file, gross_portfolio_indices, company_name, year, month)

def risk_quality_indicator(excel_file, company_name, year, month):
    risk_portfolio_indices = [
        140410, 140415, 140420, 140425, 140510, 140515, 140520, 140525,
        141110, 141115, 141120, 141125, 141210, 141215, 141220, 141225,
        144110, 144115, 144120, 144125, 144210, 144215, 144220, 144225,
        144810, 144815, 144820, 144825, 145410, 145415, 145420, 145425,
        145510, 145515, 145520, 145525, 146110, 146115, 146120, 146125,
        146210, 146215, 146220, 146225, 146910, 146915, 146920, 146925,
        146935, 146940, 146945, 146950
    ]
    qualified_portfolio = summation(excel_file, risk_portfolio_indices, company_name, year, month)
    gross_portf = gross_portfolio(excel_file,company_name, year, month)

    return qualified_portfolio / gross_portf if gross_portf != 0 else 0

def risk_quality_with_writeoffs(excel_file, company_name, year, month):
    # total qualified portfolio
    qualified_portfolio = risk_quality_indicator(excel_file, company_name, year, month)
    # writeoffs
    writeoffs = list(excel_file.loc[(excel_file['CUENTA'] == 831015) & (excel_file['fecha'] == f'{int(year)}-{int(month)}-01'),company_name])[0]
    total_with_writeoffs_indices = [
        140400, 140500, 141100, 141200, 144100, 144200, 144800,
        145400, 145500, 146100, 146200, 146900, 831015
    ]
    total_with_writeoffs = summation(excel_file, total_with_writeoffs_indices, company_name, year, month)

    return (qualified_portfolio + writeoffs) / total_with_writeoffs if total_with_writeoffs != 0 else 0

# Total Portfolio at Risk Coverage Indicator
def total_risk_coverage_indicator(excel_file, company_name, year, month):
    deterioration_indices = [140800, 144500, 145100, 145800, 146500, 146800, 147100]
    provisions = summation(excel_file, deterioration_indices, company_name, year, month)
    gross_portf = gross_portfolio(excel_file, company_name, year, month)
    return provisions / gross_portf if gross_portf != 0 else 0

# Productive asset
def productive_assets_ratio(excel_file, company_name, year, month):
    productive_assets_indices = [
        112000, 120000, 130000, 140405, 140410, 140505, 140510,
        141105, 141110, 141205, 141210, 144105, 144110, 144205,
        144210, 144805, 144810, 145405, 145410, 145505, 145510,
        146105, 146110, 146205, 146210, 146905, 146910, 146930,
        146935, 160505, 161505
    ]
    productive_assets = summation(excel_file, productive_assets_indices, company_name, year, month)
    total_assets = list(excel_file.loc[(excel_file['CUENTA'] == 100000) & (excel_file['fecha'] == f'{int(year)}-{int(month)}-01'),company_name])[0]
    return productive_assets / total_assets if total_assets != 0 else 0

# Individual Coverage Indicator of the Unproductive Portfolio for the At-Risk Portfolio
def individual_coverage_nonproductive_portfolio(excel_file, company_name, year, month):
    provisions_cde_indices = [
        140815, 140820, 140825,
        144525, 144530, 144535, 144540, 144545, 144550,
        145115, 145120, 145125,
        145825, 145830, 145835, 145840, 145845, 145850,
        146525, 146530, 146535, 146540, 146545, 146550,
        147115, 147120, 147125, 147140, 147145, 147150
    ]
    provisions = summation(excel_file, provisions_cde_indices, company_name, year, month)

    overdue_portfolio_indices = [
        140415, 140420, 140425, 140515, 140520, 140525,
        141115, 141120, 141125, 141215, 141220, 141225,
        144115, 144120, 144125, 144215, 144220, 144225,
        144815, 144820, 144825, 145415, 145420, 145425,
        145515, 145520, 145525, 146115, 146120, 146125,
        146215, 146220, 146225, 146915, 146920, 146925,
        146940, 146945, 146950
    ]
    overdue = summation(excel_file, overdue_portfolio_indices, company_name, year, month)

    return provisions / overdue if overdue != 0 else 0

### Managerial Quality

In [4]:
# Operating Financial Margin Indicator
def financial_margin_operation(excel_file, company_name, year, month):
    date_filter = f'{int(year)}-{int(month)}-01'
    
    positive_margin = list(excel_file.loc[(excel_file['CUENTA'] == 410000) & (excel_file['fecha'] == date_filter), company_name])[0]
    negative_margin1 = list(excel_file.loc[(excel_file['CUENTA'] == 610000) & (excel_file['fecha'] == date_filter), company_name])[0]
    serie = excel_file.loc[(excel_file['CUENTA'] == 700000) & (excel_file['fecha'] == date_filter), company_name]
    negative_margin2 = 0 if serie.empty else serie.iloc[0]

    sales_income = list(excel_file.loc[(excel_file['CUENTA'] == 410000) & (excel_file['fecha'] == date_filter), company_name])[0]
    
    negative_margin = negative_margin1 + negative_margin2
    return (positive_margin - negative_margin) / sales_income if sales_income != 0 else 0


# Operating Margin Indicator (usa 'summation', no se modifica)
def operational_margin(excel_file, company_name, year, month):
    income_pos_indices = [410000, 422500]
    income_neg_indices = [610000, 700000, 510500, 510700, 511000, 511500, 540000]
    sales_indices = [410000, 422500]
    income_pos = summation(excel_file, income_pos_indices, company_name, year, month)
    income_neg = summation(excel_file, income_neg_indices, company_name, year, month)
    sales_income = summation(excel_file, sales_indices, company_name, year, month)
    return (income_pos - income_neg) / sales_income if sales_income != 0 else 0

# Indicator of the relationship between financial obligations and total liabilities
def financial_obligations_ratio(excel_file, company_name, year, month):
    date_filter = f'{int(year)}-{int(month)}-01'
    
    obligations = list(excel_file.loc[(excel_file['CUENTA'] == 230000) & (excel_file['fecha'] == date_filter), company_name])[0]
    total_liabilities = list(excel_file.loc[(excel_file['CUENTA'] == 200000) & (excel_file['fecha'] == date_filter), company_name])[0]
    
    return obligations / total_liabilities if total_liabilities != 0 else 0

# Balance structure (usa 'summation', no se modifica)
def balance_structure(excel_file, company_name, year, month):
    productive_assets_indices = [
        112000, 120000, 130000, 140405, 140410, 140505, 140510,
        141105, 141110, 141205, 141210, 144105, 144110, 144205,
        144210, 144805, 144810, 145405, 145410, 145505, 145510,
        146105, 146110, 146205, 146210, 146905, 146910, 146930,
        146935, 160505, 161505
    ]
    interest_liabilities_indices = [210000, 230000]
    productive_assets = summation(excel_file, productive_assets_indices, company_name, year, month)
    interest_liabilities = summation(excel_file, interest_liabilities_indices, company_name, year, month)
    return productive_assets / interest_liabilities if interest_liabilities != 0 else 0


### Earnings Strength

In [5]:
# Return on equity indicator
def equity_indicator():
    return 1

# Net margin indicator
def net_margin_indicator(excel_file, company_name, year, month):
    net_surplus = list(excel_file.loc[(excel_file['CUENTA'] == 530000) & (excel_file['fecha'] == f'{int(year)}-{int(month)}-01'),company_name])[0]
    incomes_indices = [410000, 422500]
    total_incomes = summation(excel_file, incomes_indices, company_name, year, month)
    return net_surplus/total_incomes if total_incomes != 0 else 0

# Indicator of return on invested capital
def return_on_invested_capital(excel_file, company_name, year, month):
    value_january = list(excel_file.loc[(excel_file['CUENTA'] == 230000) & (excel_file['fecha'] == f'{int(year)}-{int(month)}-01'),company_name])[0]
    months = [y for y in range(1, 13, 1)]
    values_every_month = 0
    for m in months:
        month = list(excel_file.loc[(excel_file['CUENTA'] == 230000) & (excel_file['fecha'] == f'{int(year-1)}-{int(m)}-01'),company_name])[0]
        values_every_month += month

    media_last_year = values_every_month/12

    return (media_last_year + value_january)/2

### Liquidicy Efficiency

In [6]:
# Liquidity Risk Indicator
def liquidity_risk():
    return 1

## Create the csv for all the cooperatives

In [7]:
coops = pd.read_excel("resultados/Datos_2013_2025_cooperativas.xlsx")
coops

,CUENTA,NOMBRE CUENTA,COOPERATIVA DE LOS PROFESIONALES DE LA SALUD COASMEDAS,COOPERATIVA DE AHORRO Y CREDITO PARA EL BIENESTAR SOCIAL,COOPERATIVA PARA EL BIENESTAR SOCIAL,COOPERATIVA FINANCIERA SAN FRANCISCO,COOPERATIVA MULTIACTIVA DE LA AVIACION CIVIL COLOMBIANA,COOPERATIVA DE PROFESORES DE LA U NACIONAL DE COLOMBIA,CAJA COOPERATIVA CREDICOOP,COOPERATIVA DE AHORRO Y CREDITO DE SURAMERICA,...,COOPERATIVA DE AHORRO Y CREDITO COOMPARTIR,COOPERATIVA DE AHORRO Y CREDITO BERLIN,COOPERATIVA DE AHORRO Y CREDITO DE AIPE,COOPERATIVA DE AHORRO Y CREDITO DE SANTANDER LIMITADA,COOPERATIVA BELEN AHORRO Y CREDITO,COOPERATIVA DE AHORRO Y CREDITO FINANCIAFONDOS,COOPERATIVA DE AHORRO Y CREDITO DE DROGUISTAS DETALLISTAS,COOPERATIVA DE AHORRO Y CREDITO COLANTA,COOPERATIVA ESPECIALIZADA DE AHORRO Y CREDITO AFROAMERICANA,fecha
0,100000,ACTIVO,3.387150e+11,1.560676e+11,9.879448e+10,1.064674e+10,7.021719e+10,1.417133e+11,8.491366e+10,2.198054e+10,...,4.312265e+09,4.443368e+10,2.850488e+10,1.641308e+12,3.832060e+11,1.559309e+10,2.597647e+11,3.174706e+11,7.498950e+09,2022-12-01
1,110000,EFECTIVO Y EQUIVALENTE AL EFECTIVO,1.566943e+10,2.669495e+10,1.918907e+09,1.552585e+09,4.685907e+09,1.434015e+10,6.147909e+09,2.294268e+09,...,1.192062e+08,2.639965e+09,2.223880e+09,2.443691e+11,7.403156e+09,2.817348e+08,3.586048e+10,2.764945e+10,5.507006e+08,2022-12-01
2,110500,CAJA,8.229352e+08,1.100000e+06,3.589941e+08,1.718631e+08,1.241549e+08,6.995092e+08,9.522290e+07,1.118834e+07,...,2.765270e+07,2.401334e+08,2.473210e+08,1.236956e+10,2.169476e+09,0.000000e+00,2.000000e+06,5.735745e+09,4.744467e+08,2022-12-01
3,110505,CAJA GENERAL,8.180852e+08,0.000000e+00,3.589941e+08,1.718631e+08,1.146549e+08,6.995092e+08,9.522290e+07,1.048834e+07,...,2.732270e+07,2.394334e+08,2.473210e+08,1.235737e+10,2.168001e+09,0.000000e+00,0.000000e+00,5.732195e+09,4.738467e+08,2022-12-01
4,110510,CAJA MENOR,4.850000e+06,1.100000e+06,0.000000e+00,0.000000e+00,9.500000e+06,0.000000e+00,0.000000e+00,7.000000e+05,...,3.300000e+05,7.000000e+05,0.000000e+00,1.218300e+07,1.475000e+06,0.000000e+00,2.000000e+06,3.550000e+06,6.000000e+05,2022-12-01
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
292668,935000,OTRAS ACREEDORAS DE CONTROL,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,9.900000e+01,0.000000e+00,0.000000e+00,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.934962e+08,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,2014-12-01
292669,960000,ACREEDORAS CONTINGENTES POR CONTRA,2.645291e+11,1.493243e+08,5.438012e+10,2.747754e+09,4.058720e+10,2.363426e+11,9.935065e+10,1.808016e+10,...,8.768710e+09,4.080666e+10,9.539936e+09,1.535086e+12,1.871075e+11,9.630938e+09,2.930567e+11,2.023843e+11,6.737913e+09,2014-12-01
292670,960500,ACREEDORAS CONTINGENTES POR CONTRA (DB),2.645291e+11,1.493243e+08,5.438012e+10,2.747754e+09,4.058720e+10,2.363426e+11,9.935065e+10,1.808016e+10,...,8.768710e+09,4.080666e+10,9.539936e+09,1.535086e+12,1.871075e+11,9.630938e+09,2.930567e+11,2.023843e+11,6.737913e+09,2014-12-01
292671,980000,ACREEDORAS DE CONTROL POR CONTRA,2.619920e+10,3.159206e+11,7.700000e+09,1.299760e+09,6.160000e+09,0.000000e+00,1.232000e+10,3.080000e+09,...,1.274480e+09,3.654957e+09,1.386000e+09,8.014403e+10,8.129557e+09,1.289659e+09,5.000000e+09,2.140509e+10,2.266800e+09,2014-12-01


Generate new columns for the DataFrame

In [9]:
keys = list(coops.columns)
keys.remove("CUENTA")
keys.remove('NOMBRE CUENTA')
keys.remove('fecha')
print(keys)

['COOPERATIVA DE LOS PROFESIONALES DE LA SALUD COASMEDAS', 'COOPERATIVA DE AHORRO Y CREDITO PARA EL BIENESTAR SOCIAL', 'COOPERATIVA PARA EL BIENESTAR SOCIAL', 'COOPERATIVA FINANCIERA SAN FRANCISCO', 'COOPERATIVA MULTIACTIVA DE LA AVIACION CIVIL COLOMBIANA', 'COOPERATIVA DE PROFESORES DE LA U NACIONAL DE COLOMBIA', 'CAJA COOPERATIVA CREDICOOP', 'COOPERATIVA DE AHORRO Y CREDITO DE SURAMERICA', 'FINANCIERA COOPERATIVA COLOMBIANA DE INGENIEROS', 'COOPERATIVA DEL MAGISTERIO', 'COOPERATIVA DE AHORRO Y CREDITO DE CHIPAQUE', 'COOPERATIVA DE AHORRO Y CREDITO UNIVERSIDAD SANTO TOMAS', 'CAJA COOPERATIVA PETROLERA', 'COOPERATIVA TEXAS LTDA', 'COOPERATIVA DE LOS TRABAJADORES DEL INSTITUTO DE LOS SEGUROS SOCIALES', 'COOPERATIVA DE TRABAJADORES DE BAVARIA DIRECCION Y VENTAS LTDA', 'COPROCENVA COOPERATIVA DE AHORRO Y CREDITO', 'COOPERATIVA DE AHORRO Y CREDITO DE TENJO', 'COOPERATIVA DE TRABAJADORES Y PENSIONADOS DE LA E.A.A.B.', 'COPERATIVA INDEPENDIENTE DE EMPLEADOS DE ANTIOQUIA', 'COOPERATIVA MULTIA

In [15]:
from concurrent.futures import ThreadPoolExecutor
from functools import partial

In [11]:
indexes_camel = ['Quebranto Patrimonial', 'Relación Solvencia', 'Relación entre Aportes sociales mínimos no reducibles y Capital Social', 'Relación entre el Capital Institucional y el Activo Total', 
                 'Indicador de calidad por riesgo', 'Indicador de calidad por riesgo con castigos', 'Indicador de Cobertura de la Cartera Total en Riesgo', 'Activo Productivo', 'Indicador de Cobertura individual de la cartera improductiva para la cartera en Riesgo',
                 'Indicador de Margen Financiero de Operación', 'Indicador de Margen Operacional', 'Indicador de relación entre las obligaciones financieras y el pasivo total', 'Estructura de Balance',
                 'Indicador de rentabilidad sobre recursos propios - ROE', 'Indicador de margen neto', 'Indicador de rentabilidad sobre el capital invertido - ROIC',
                 'Indicador de Riesgo de Liquidez - IRL']
years = range(2014, 2025)
months = range(1, 13)

We need to parallelize the next cells, because there is a lot of calculations there (for the big dataset)

In [14]:
def calculate_values_capital(company, coops, years, months, indices_camel):
    rows = []
    for y in years:
        for m in months:
            c1 = loss_of_assets(coops, company, y, m)
            c2 = solvency_ratio()
            c3 = min_social_contrib_social_capital(coops, company, y, m)
            c4 = capital_contribution_ratio(coops, company, y, m)

            rows.append([y, m, indices_camel[0], company, c1])
            rows.append([y, m, indices_camel[1], company, c2])
            rows.append([y, m, indices_camel[2], company, c3])
            rows.append([y, m, indices_camel[3], company, c4])
    return rows

# parameters
partial_func_c = partial(calculate_values_capital, coops=coops, years=years, months=months, indices_camel=indexes_camel)

# execute for every company
with ProcessPoolExecutor() as executor:
    resultados = executor.map(partial_func_c, keys)

# organize all the info in a dataframe
rows_c = [fila for sublista in resultados for fila in sublista]

df = pd.DataFrame(rows_c, columns=['Año', 'Mes', 'índice CAMEL', 'Cooperativa', 'valor CAMEL'])
df_pivot = df.pivot_table(index=['Año', 'Mes', 'índice CAMEL'], columns='Cooperativa', values='valor CAMEL').reset_index()
df_pivot.columns.name = None

In [12]:
df_pivot

,Año,Mes,índice CAMEL,CAJA COOPERATIVA CREDICOOP,CAJA COOPERATIVA PETROLERA,CASA NACIONAL DEL PROFESOR,COOPANTEX COOPERATIVA DE AHORRO Y CREDITO,COOPERATIVA AHORRO Y CREDITO GOMEZ PLATA LTDA.,COOPERATIVA BELEN AHORRO Y CREDITO,COOPERATIVA CALDENSE DEL PROFESOR,...,COOPERATIVA SAN VICENTE DE PAUL LTDA.,COOPERATIVA TELEPOSTAL LTDA,COOPERATIVA TEXAS LTDA,COPERATIVA INDEPENDIENTE DE EMPLEADOS DE ANTIOQUIA,COPERATIVA MULTIACTIVA DE EDUCADORES DE BOYACA,COPROCENVA COOPERATIVA DE AHORRO Y CREDITO,EMPRESA COOPERATIVA DE AHORRO Y CREDITO SIGLO XX LTDA.,FINANCIERA COOPERATIVA COLOMBIANA DE INGENIEROS,GRAN COOPERATIVA DE ENERGIA ELECTRICA Y RECURSOS NATURALES,"MULTIACTIVA EL ROBLE, ENTIDAD COOPERATIVA"
0,2014,1,Quebranto Patrimonial,1.500449,1.310736,1.188087,1.588034,1.481415,2.088982,0.000000,...,1.366898,1.575925,1.430989,1.415437,1.119304,1.367164,1.328639,1.615208,1.634529,2.159444
1,2014,1,Relación Solvencia,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
2,2014,1,Relación entre Aportes sociales mínimos no red...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,2014,1,Relación entre el Capital Institucional y el A...,0.145260,0.057027,0.047077,0.126778,0.085107,0.094572,0.000000,...,0.072971,0.157531,0.112465,0.066512,0.031398,0.060938,0.152577,0.116553,0.243769,0.256789
4,2014,2,Quebranto Patrimonial,1.508398,1.310754,1.190007,1.597500,1.480301,2.099399,1.001952,...,1.367030,1.581733,1.440119,1.415271,1.117340,1.370708,1.337403,1.625810,1.637438,2.173214
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
523,2024,11,Relación entre el Capital Institucional y el A...,0.077386,0.076594,0.026661,0.038778,0.057028,0.049453,0.079219,...,0.018091,0.225168,0.119796,0.078052,0.017746,0.029156,0.096079,0.172138,0.180172,0.211573
524,2024,12,Quebranto Patrimonial,1.336764,1.328436,1.148136,1.312731,1.411058,1.431589,1.392271,...,0.967298,1.625325,1.302999,1.594671,1.163527,1.159258,1.388815,1.880173,1.586720,1.700891
525,2024,12,Relación Solvencia,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
526,2024,12,Relación entre Aportes sociales mínimos no red...,0.913576,0.757703,0.632018,0.697214,0.564119,1.000000,0.647619,...,0.914066,0.585316,0.257352,0.952096,0.520750,0.385418,0.858692,0.855241,0.343540,0.139353


In [16]:
def calculate_values_assets(company, coops, years, months, indices_camel):
    rows = []
    for company in keys:
        for y in years:
            for m in months:
                # values for A: ASSETS
                a1 = risk_quality_indicator(coops, company, y, m)
                a2 = risk_quality_with_writeoffs(coops, company, y, m)
                a3 = total_risk_coverage_indicator(coops, company, y, m)
                a4 = productive_assets_ratio(coops, company, y, m)
                a5 = individual_coverage_nonproductive_portfolio(coops, company, y, m)

                rows.append([y, m, indices_camel[4], company, a1])
                rows.append([y, m, indices_camel[5], company, a2])
                rows.append([y, m, indices_camel[6], company, a3])
                rows.append([y, m, indices_camel[7], company, a4])
                rows.append([y, m, indices_camel[8], company, a5])
    return rows

# parameters
partial_func_a = partial(calculate_values_assets, coops=coops, years=years, months=months, indices_camel=indexes_camel)

# execute for every company
with ThreadPoolExecutor() as executor:
    resultados = executor.map(partial_func_a, keys)

# organize all the info in a dataframe
rows_a = [fila for sublista in resultados for fila in sublista]

df2 = pd.DataFrame(rows_a, columns=['Año', 'Mes', 'índice CAMEL', 'Cooperativa', 'valor CAMEL'])
df_pivot2 = df2.pivot_table(index=['Año', 'Mes', 'índice CAMEL'], columns='Cooperativa', values='valor CAMEL').reset_index()
df_pivot2.columns.name = None        

KeyboardInterrupt: 

In [21]:
df_pivot2

,Año,Mes,índice CAMEL,CAJA COOPERATIVA CREDICOOP,CAJA COOPERATIVA PETROLERA,CASA NACIONAL DEL PROFESOR,COOPANTEX COOPERATIVA DE AHORRO Y CREDITO,COOPERATIVA AHORRO Y CREDITO GOMEZ PLATA LTDA.,COOPERATIVA BELEN AHORRO Y CREDITO,COOPERATIVA CALDENSE DEL PROFESOR,...,COOPERATIVA SAN VICENTE DE PAUL LTDA.,COOPERATIVA TELEPOSTAL LTDA,COOPERATIVA TEXAS LTDA,COPERATIVA INDEPENDIENTE DE EMPLEADOS DE ANTIOQUIA,COPERATIVA MULTIACTIVA DE EDUCADORES DE BOYACA,COPROCENVA COOPERATIVA DE AHORRO Y CREDITO,EMPRESA COOPERATIVA DE AHORRO Y CREDITO SIGLO XX LTDA.,FINANCIERA COOPERATIVA COLOMBIANA DE INGENIEROS,GRAN COOPERATIVA DE ENERGIA ELECTRICA Y RECURSOS NATURALES,"MULTIACTIVA EL ROBLE, ENTIDAD COOPERATIVA"
0,2014,1,balance_structure,2.001510e+00,1.386005e+00,1.323835e+00,1.286730e+00,1.207956e+00,1.211381e+00,0.000000,...,1.511317e+00,2.828472,1.873193,1.127042e+00,1.267051e+00,1.264017e+00,2.452405e+00,1.352501e+00,4.447264,1.918250
1,2014,1,capital_contribution_ratio,1.452603e-01,5.702671e-02,4.707728e-02,1.267778e-01,8.510686e-02,9.457200e-02,0.000000,...,7.297070e-02,0.157531,0.112465,6.651162e-02,3.139802e-02,6.093769e-02,1.525774e-01,1.165531e-01,0.243769,0.256789
2,2014,1,equity_indicator,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,...,1.000000e+00,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000
3,2014,1,financial_margin_operation,8.549985e-01,7.660669e-01,7.351898e-01,6.622011e-01,7.931703e-01,7.769939e-01,0.000000,...,7.674119e-01,0.893440,0.779820,7.039941e-01,7.191271e-01,7.861258e-01,8.580408e-01,7.896245e-01,0.932666,0.842562
4,2014,1,financial_obligations_ratio,1.243085e-01,2.486871e-01,6.844926e-03,7.481717e-02,0.000000e+00,0.000000e+00,0.000000,...,1.452490e-01,0.000000,0.000744,1.272631e-02,1.237561e-02,5.222927e-02,2.408727e-01,5.649955e-03,0.004337,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2239,2024,12,return_on_invested_capital,3.391787e+09,2.250769e+10,7.860631e+09,1.590581e+09,4.379361e+08,3.660149e+09,0.000000,...,1.266983e+08,343773.416667,66282.041667,1.519239e+09,1.065696e+09,1.381704e+10,2.544307e+08,5.680970e+07,9946.458333,0.000000
2240,2024,12,risk_quality_indicator,9.950384e-02,9.279456e-02,2.168472e-02,2.268436e-01,9.901056e-02,1.441177e-01,0.045651,...,9.901022e-02,0.107197,0.081108,5.047547e-02,1.913314e-02,4.154364e-02,5.114339e-02,1.309176e-02,0.065909,0.066153
2241,2024,12,risk_quality_with_writeoffs,1.258836e-01,1.139508e-01,5.320317e-03,5.389335e-02,1.694322e-02,7.903804e-02,0.021841,...,3.082890e-01,0.040793,0.003805,8.731617e-02,2.515740e-03,1.780211e-02,6.096142e-02,2.567755e-01,0.005773,0.004604
2242,2024,12,solvency_ratio,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,...,1.000000e+00,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000


In [13]:
def calculate_values_managerial(company, coops, years, months, indices_camel):
    rows = []
    for company in keys:
        for y in years:
            for m in months:
                # values for M: MANAGERIAL
                m1 = financial_margin_operation(coops, company, y, m)
                m2 = operational_margin(coops, company, y, m)
                m3 = financial_obligations_ratio(coops, company, y, m)
                m4 = balance_structure(coops, company, y, m)

                rows.append([y, m, indices_camel[9], company, m1])
                rows.append([y, m, indices_camel[10], company, m2])
                rows.append([y, m, indices_camel[11], company, m3])
                rows.append([y, m, indices_camel[12], company, m4])

# parameters
partial_func_m = partial(calculate_values_managerial, coops=coops, years=years, months=months, indices_camel=indexes_camel)

# execute for every company
with ProcessPoolExecutor() as executor:
    resultados = executor.map(partial_func_m, keys)

# organize all the info in a dataframe
rows_m = [fila for sublista in resultados for fila in sublista]

df3 = pd.DataFrame(rows_m, columns=['Año', 'Mes', 'índice CAMEL', 'Cooperativa', 'valor CAMEL'])

df_pivot3 = df3.pivot_table(index=['Año', 'Mes', 'índice CAMEL'], columns='Cooperativa', values='valor CAMEL').reset_index()

df_pivot3.columns.name = None 

KeyboardInterrupt: 

In [23]:
df_pivot3

,Año,Mes,índice CAMEL,CAJA COOPERATIVA CREDICOOP,CAJA COOPERATIVA PETROLERA,CASA NACIONAL DEL PROFESOR,COOPANTEX COOPERATIVA DE AHORRO Y CREDITO,COOPERATIVA AHORRO Y CREDITO GOMEZ PLATA LTDA.,COOPERATIVA BELEN AHORRO Y CREDITO,COOPERATIVA CALDENSE DEL PROFESOR,...,COOPERATIVA SAN VICENTE DE PAUL LTDA.,COOPERATIVA TELEPOSTAL LTDA,COOPERATIVA TEXAS LTDA,COPERATIVA INDEPENDIENTE DE EMPLEADOS DE ANTIOQUIA,COPERATIVA MULTIACTIVA DE EDUCADORES DE BOYACA,COPROCENVA COOPERATIVA DE AHORRO Y CREDITO,EMPRESA COOPERATIVA DE AHORRO Y CREDITO SIGLO XX LTDA.,FINANCIERA COOPERATIVA COLOMBIANA DE INGENIEROS,GRAN COOPERATIVA DE ENERGIA ELECTRICA Y RECURSOS NATURALES,"MULTIACTIVA EL ROBLE, ENTIDAD COOPERATIVA"
0,2014,1,balance_structure,2.001510,1.386005,1.323835,1.286730,1.207956,1.211381,0.0,...,1.511317,2.828472,1.873193,1.127042,1.267051,1.264017,2.452405,1.352501,4.447264,1.918250
1,2014,1,capital_contribution_ratio,0.145260,0.057027,0.047077,0.126778,0.085107,0.094572,0.0,...,0.072971,0.157531,0.112465,0.066512,0.031398,0.060938,0.152577,0.116553,0.243769,0.256789
2,2014,1,financial_margin_operation,0.854999,0.766067,0.735190,0.662201,0.793170,0.776994,0.0,...,0.767412,0.893440,0.779820,0.703994,0.719127,0.786126,0.858041,0.789625,0.932666,0.842562
3,2014,1,financial_obligations_ratio,0.124309,0.248687,0.006845,0.074817,0.000000,0.000000,0.0,...,0.145249,0.000000,0.000744,0.012726,0.012376,0.052229,0.240873,0.005650,0.004337,0.000000
4,2014,1,individual_coverage_nonproductive_portfolio,0.000000,0.000445,NaN,NaN,0.000000,NaN,NaN,...,0.000000,0.000000,0.000000,0.000000,NaN,0.060056,0.000000,0.197307,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1711,2024,12,productive_assets_ratio,0.886718,0.881181,NaN,NaN,0.882883,NaN,NaN,...,0.767313,0.818121,0.894986,0.928845,NaN,0.886829,0.877507,0.850814,0.913173,NaN
1712,2024,12,risk_quality_indicator,0.099504,0.092795,NaN,NaN,0.099011,NaN,NaN,...,0.099010,0.107197,0.081108,0.050475,NaN,0.041544,0.051143,0.013092,0.065909,NaN
1713,2024,12,risk_quality_with_writeoffs,0.125884,0.113951,NaN,NaN,0.016943,NaN,NaN,...,0.308289,0.040793,0.003805,0.087316,NaN,0.017802,0.060961,0.256776,0.005773,NaN
1714,2024,12,solvency_ratio,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.0,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


In [ ]:
def calculate_values_earnings(company, coops, years, months, indices_camel):
    rows = []
    for company in keys:
        for y in years:
            for m in months:
                # values for E: EARNINGS
                e1 = equity_indicator()
                e2 = net_margin_indicator(coops, company, y, m)
                e3 = return_on_invested_capital(coops, company, y, m)

                rows.append([y, m, indices_camel[13], company, e1])
                rows.append([y, m, indices_camel[14], company, e2])
                rows.append([y, m, indices_camel[15], company, e3])

# parameters
partial_func_e = partial(calculate_values_earnings, coops=coops, years=years, months=months, indices_camel=indexes_camel)

# execute for every company
with ProcessPoolExecutor() as executor:
    resultados = executor.map(partial_func_e, keys)

# organize all the info in a dataframe
rows_e = [fila for sublista in resultados for fila in sublista]

df4 = pd.DataFrame(rows_e, columns=['Año', 'Mes', 'índice CAMEL', 'Cooperativa', 'valor CAMEL'])

df_pivot4 = df4.pivot_table(index=['Año', 'Mes', 'índice CAMEL'], columns='Cooperativa', values='valor CAMEL').reset_index()

df_pivot4.columns.name = None

In [17]:
df_pivot4

,Año,Mes,índice CAMEL,CAJA COOPERATIVA CREDICOOP,CAJA COOPERATIVA PETROLERA,CASA NACIONAL DEL PROFESOR,COOPANTEX COOPERATIVA DE AHORRO Y CREDITO,COOPERATIVA AHORRO Y CREDITO GOMEZ PLATA LTDA.,COOPERATIVA BELEN AHORRO Y CREDITO,COOPERATIVA CALDENSE DEL PROFESOR,...,COOPERATIVA SAN VICENTE DE PAUL LTDA.,COOPERATIVA TELEPOSTAL LTDA,COOPERATIVA TEXAS LTDA,COPERATIVA INDEPENDIENTE DE EMPLEADOS DE ANTIOQUIA,COPERATIVA MULTIACTIVA DE EDUCADORES DE BOYACA,COPROCENVA COOPERATIVA DE AHORRO Y CREDITO,EMPRESA COOPERATIVA DE AHORRO Y CREDITO SIGLO XX LTDA.,FINANCIERA COOPERATIVA COLOMBIANA DE INGENIEROS,GRAN COOPERATIVA DE ENERGIA ELECTRICA Y RECURSOS NATURALES,"MULTIACTIVA EL ROBLE, ENTIDAD COOPERATIVA"
0,2014,1,balance_structure,2.001510e+00,1.386005e+00,1.323835e+00,1.286730e+00,1.207956e+00,1.211381e+00,0.0,...,1.511317e+00,2.828472,1.873193,1.127042e+00,1.267051e+00,1.264017e+00,2.452405e+00,1.352501e+00,4.447264,1.918250
1,2014,1,capital_contribution_ratio,1.452603e-01,5.702671e-02,4.707728e-02,1.267778e-01,8.510686e-02,9.457200e-02,0.0,...,7.297070e-02,0.157531,0.112465,6.651162e-02,3.139802e-02,6.093769e-02,1.525774e-01,1.165531e-01,0.243769,0.256789
2,2014,1,equity_indicator,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.0,...,1.000000e+00,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000
3,2014,1,financial_margin_operation,8.549985e-01,7.660669e-01,7.351898e-01,6.622011e-01,7.931703e-01,7.769939e-01,0.0,...,7.674119e-01,0.893440,0.779820,7.039941e-01,7.191271e-01,7.861258e-01,8.580408e-01,7.896245e-01,0.932666,0.842562
4,2014,1,financial_obligations_ratio,1.243085e-01,2.486871e-01,6.844926e-03,7.481717e-02,0.000000e+00,0.000000e+00,0.0,...,1.452490e-01,0.000000,0.000744,1.272631e-02,1.237561e-02,5.222927e-02,2.408727e-01,5.649955e-03,0.004337,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2107,2024,12,return_on_invested_capital,3.391787e+09,2.250769e+10,7.860631e+09,1.590581e+09,4.379361e+08,3.660149e+09,0.0,...,1.266983e+08,343773.416667,66282.041667,1.519239e+09,1.065696e+09,1.381704e+10,2.544307e+08,5.680970e+07,9946.458333,0.000000
2108,2024,12,risk_quality_indicator,9.950384e-02,9.279456e-02,NaN,NaN,9.901056e-02,NaN,NaN,...,9.901022e-02,0.107197,0.081108,5.047547e-02,NaN,4.154364e-02,5.114339e-02,1.309176e-02,0.065909,NaN
2109,2024,12,risk_quality_with_writeoffs,1.258836e-01,1.139508e-01,NaN,NaN,1.694322e-02,NaN,NaN,...,3.082890e-01,0.040793,0.003805,8.731617e-02,NaN,1.780211e-02,6.096142e-02,2.567755e-01,0.005773,NaN
2110,2024,12,solvency_ratio,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.0,...,1.000000e+00,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000


In [ ]:
def calculate_values_liquidicy(company, coops, years, months, indices_camel):
    rows = []
    for company in keys:
        for y in years:
            for m in months:
                # values for L: LIQUIDICY
                l1 = liquidity_risk()
                rows.append([y, m, indices_camel[16], company, l1])

# parameters
partial_func_l = partial(calculate_values_earnings, coops=coops, years=years, months=months, indices_camel=indexes_camel)

# execute for every company
with ProcessPoolExecutor() as executor:
    resultados = executor.map(partial_func_l, keys)

# organize all the info in a dataframe
rows_l = [fila for sublista in resultados for fila in sublista]

df5 = pd.DataFrame(rows_l, columns=['Año', 'Mes', 'índice CAMEL', 'Cooperativa', 'valor CAMEL'])

df_pivot5 = df5.pivot_table(index=['Año', 'Mes', 'índice CAMEL'], columns='Cooperativa', values='valor CAMEL').reset_index()

df_pivot5.columns.name = None

In [19]:
df_pivot5

,Año,Mes,índice CAMEL,CAJA COOPERATIVA CREDICOOP,CAJA COOPERATIVA PETROLERA,CASA NACIONAL DEL PROFESOR,COOPANTEX COOPERATIVA DE AHORRO Y CREDITO,COOPERATIVA AHORRO Y CREDITO GOMEZ PLATA LTDA.,COOPERATIVA BELEN AHORRO Y CREDITO,COOPERATIVA CALDENSE DEL PROFESOR,...,COOPERATIVA SAN VICENTE DE PAUL LTDA.,COOPERATIVA TELEPOSTAL LTDA,COOPERATIVA TEXAS LTDA,COPERATIVA INDEPENDIENTE DE EMPLEADOS DE ANTIOQUIA,COPERATIVA MULTIACTIVA DE EDUCADORES DE BOYACA,COPROCENVA COOPERATIVA DE AHORRO Y CREDITO,EMPRESA COOPERATIVA DE AHORRO Y CREDITO SIGLO XX LTDA.,FINANCIERA COOPERATIVA COLOMBIANA DE INGENIEROS,GRAN COOPERATIVA DE ENERGIA ELECTRICA Y RECURSOS NATURALES,"MULTIACTIVA EL ROBLE, ENTIDAD COOPERATIVA"
0,2014,1,balance_structure,2.001510e+00,1.386005e+00,1.323835e+00,1.286730e+00,1.207956e+00,1.211381e+00,0.0,...,1.511317e+00,2.828472,1.873193,1.127042e+00,1.267051e+00,1.264017e+00,2.452405e+00,1.352501e+00,4.447264,1.918250
1,2014,1,capital_contribution_ratio,1.452603e-01,5.702671e-02,4.707728e-02,1.267778e-01,8.510686e-02,9.457200e-02,0.0,...,7.297070e-02,0.157531,0.112465,6.651162e-02,3.139802e-02,6.093769e-02,1.525774e-01,1.165531e-01,0.243769,0.256789
2,2014,1,equity_indicator,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.0,...,1.000000e+00,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000
3,2014,1,financial_margin_operation,8.549985e-01,7.660669e-01,7.351898e-01,6.622011e-01,7.931703e-01,7.769939e-01,0.0,...,7.674119e-01,0.893440,0.779820,7.039941e-01,7.191271e-01,7.861258e-01,8.580408e-01,7.896245e-01,0.932666,0.842562
4,2014,1,financial_obligations_ratio,1.243085e-01,2.486871e-01,6.844926e-03,7.481717e-02,0.000000e+00,0.000000e+00,0.0,...,1.452490e-01,0.000000,0.000744,1.272631e-02,1.237561e-02,5.222927e-02,2.408727e-01,5.649955e-03,0.004337,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2239,2024,12,return_on_invested_capital,3.391787e+09,2.250769e+10,7.860631e+09,1.590581e+09,4.379361e+08,3.660149e+09,0.0,...,1.266983e+08,343773.416667,66282.041667,1.519239e+09,1.065696e+09,1.381704e+10,2.544307e+08,5.680970e+07,9946.458333,0.000000
2240,2024,12,risk_quality_indicator,9.950384e-02,9.279456e-02,NaN,NaN,9.901056e-02,NaN,NaN,...,9.901022e-02,0.107197,0.081108,5.047547e-02,NaN,4.154364e-02,5.114339e-02,1.309176e-02,0.065909,NaN
2241,2024,12,risk_quality_with_writeoffs,1.258836e-01,1.139508e-01,NaN,NaN,1.694322e-02,NaN,NaN,...,3.082890e-01,0.040793,0.003805,8.731617e-02,NaN,1.780211e-02,6.096142e-02,2.567755e-01,0.005773,NaN
2242,2024,12,solvency_ratio,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.0,...,1.000000e+00,1.000000,1.000000,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000e+00,1.000000,1.000000
